# Lightweight Authentication Protocol – Registration Phase

This notebook implements a lightweight authentication and key agreement protocol between a Smart Meter (SMi) and Service Provider (SPj). It simulates both the **registration and mutual authentication phases** using hashing, AES encryption, and XOR operations. 

#### Referenced Research Paper:  
["A lightweight authentication scheme with privacy protection for smart grid communications"](https://doi.org/10.1016/j.future.2019.05.069)


In [11]:
import hashlib
import secrets
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64

def h(data: bytes) -> bytes:
    return hashlib.sha256(data).digest()

def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))

def xor_repeat(data: bytes, key: bytes) -> bytes:
    return bytes(data[i] ^ key[i % len(key)] for i in range(len(data)))

# ------------------- Service Provider (SPj) -------------------

class ServiceProvider:
    def __init__(self, IDj: str, master_key: str):
        self.IDj = IDj.encode()
        self.master_key = master_key
        self.database = {}
        self.aes_key = get_random_bytes(16)
        self.id_len = None
        self.r1_len = None
        self.step4_state = {}

    def encrypt(self, plaintext: bytes) -> str:
        cipher = AES.new(self.aes_key, AES.MODE_CBC)
        iv = cipher.iv
        padded = pad(plaintext, AES.block_size)
        encrypted = cipher.encrypt(padded)
        return base64.b64encode(iv + encrypted).decode()

    def decrypt(self, ciphertext_b64: str) -> bytes:
        raw = base64.b64decode(ciphertext_b64)
        iv = raw[:16]
        ciphertext = raw[16:]
        cipher = AES.new(self.aes_key, AES.MODE_CBC, iv)
        decrypted = unpad(cipher.decrypt(ciphertext), AES.block_size)
        return decrypted

    def process_registration(self, IDi: str, r1: bytes) -> str:
        IDi_bytes = IDi.encode()
        h_IDj_s = h(self.IDj + self.master_key)
        xor1 = xor_bytes(IDi_bytes, h_IDj_s[:len(IDi_bytes)])
        xor2 = xor_repeat(r1, IDi_bytes)
        combined = xor1 + xor2
        Mi = self.encrypt(combined)
        Qi = h(xor_bytes(IDi_bytes + self.IDj, self.master_key) + r1)
        self.database[IDi] = Qi
        self.id_len = len(IDi_bytes)
        self.r1_len = len(r1)
        return Mi

    def process_authentication(self, Mi: str, Xi: bytes) -> str:
        print("\n[Step 2: Service Provider Processing Authentication Request]")
        decrypted = self.decrypt(Mi)
        part1 = decrypted[:self.id_len]
        part2 = decrypted[self.id_len:self.id_len + self.r1_len]

        h_IDj_s = h(self.IDj + self.master_key)[:len(part1)]
        IDi_star = xor_bytes(part1, h_IDj_s)
        r1_star = xor_repeat(part2, IDi_star)

        try:
            IDi_str = IDi_star.decode()
        except UnicodeDecodeError:
            IDi_str = IDi_star.hex()

        print(f"Recovered IDi*: {IDi_str}")
        print(f"Recovered r1*: {r1_star.hex()}")

        Qi_prime = h(xor_bytes(IDi_star + self.IDj, self.master_key) + r1_star)
        db_qi = self.database.get(IDi_str)

        if db_qi != Qi_prime:
            print("Authentication Failed: Qi mismatch.")
            return None

        print("Authentication Passed: Qi matched.")

        r2_prime = xor_bytes(Xi, h(IDi_star + r1_star)[:len(Xi)])
        print(f"Recovered r2': {r2_prime.hex()}")

        xor1 = xor_bytes(IDi_star, h(self.IDj + self.master_key)[:len(IDi_star)])
        xor2 = xor_repeat(r2_prime, IDi_star)
        Mi_prime = self.encrypt(xor1 + xor2)

        r3 = get_random_bytes(len(r2_prime))
        print(f"Generated r3: {r3.hex()}")

        k = h(xor_bytes(xor_bytes(IDi_star, r1_star), r2_prime))

        part1 = xor_bytes(h(xor_bytes(IDi_star, r2_prime) + r1_star), r3)
        part2 = h(IDi_star + r1_star + r2_prime)
        plaintext = part1 + part2 + Mi_prime.encode()
        auth_cipher = AES.new(k[:16], AES.MODE_CBC)
        auth_iv = auth_cipher.iv
        auth_encrypted = auth_cipher.encrypt(pad(plaintext, AES.block_size))
        Authji = base64.b64encode(auth_iv + auth_encrypted).decode()

        SKsp = h(IDi_star + r1_star + r2_prime + r3)
        print(f"Session key (SKsp): {SKsp.hex()}")
        print(f"Sending Authji to Smart Meter:\n{Authji}\n")

        # Save for step 4
        self.step4_state = {
            "IDi": IDi_str,
            "IDi_star": IDi_star,
            "r1_star": r1_star,
            "r2_prime": r2_prime,
            "r3": r3,
            "SKsp": SKsp,
            "Mi_prime": Mi_prime
        }

        return Authji

    def verify_authij(self, Authij: str):
        print("[Step 4: Service Provider Finalizes Authentication]")
        s = self.step4_state
        expected_authij = h(s["SKsp"] + s["r3"]).hex()

        if Authij != expected_authij:
            print("Authij verification failed! Terminating session.")
            return False

        print("Authij verified successfully.")

        xor_val = xor_bytes(s["IDi_star"] + self.IDj, self.master_key)
        Qinew = h(xor_val + s["r2_prime"])

        old_Qi = self.database.get(s["IDi"])
        self.database[s["IDi"]] = Qinew
        self.database[f"{s['IDi']}_prev"] = old_Qi

        print(f"Dynamic verification table updated for {s['IDi']}.")
        print(f"New Qi (Qinew): {Qinew.hex()}")
        print(f"Old Qi stored as Qio: {old_Qi.hex() if old_Qi else None}")

        Ackji = h(xor_bytes(s["r2_prime"], s["r3"]) + s["r1_star"]).hex()
        print(f"Ackji to be sent to Smart Meter:\n{Ackji}\n")
        return Ackji, s["r3"], s["IDi"], s["IDi_star"], s["r1_star"], s["r2_prime"], s["SKsp"], s["Mi_prime"]



# ------------------- Smart Meter (SMi) -------------------

class SmartMeter:
    def __init__(self, IDi: str):
        self.IDi = IDi
        self.IDi_bytes = IDi.encode()
        self.r1 = self.generate_random_bytes()
        self.Mi = None
        self.r2 = None
        self.Xi = None

    def generate_random_bytes(self, length: int = 8) -> bytes:
        return secrets.token_bytes(length)

    def initiate_registration(self, SPj: ServiceProvider):
        self.Mi = SPj.process_registration(self.IDi, self.r1)
        print("Registration complete for Smart Meter:", self.IDi)
        print("Stored in Tamper Resistant Device:")
        print({"IDi": self.IDi, "r1": self.r1.hex(), "Mi": self.Mi})

    def initiate_authentication(self):
        self.r2 = self.generate_random_bytes()
        hash_val = h(self.IDi_bytes + self.r1)
        self.Xi = xor_bytes(self.r2, hash_val[:len(self.r2)])
        print("\n[Step 1: Authentication Initiated]")
        print(f"Generated r2: {self.r2.hex()}")
        print(f"Computed Xi = r2 ⊕ h(IDi || r1): {self.Xi.hex()}")
        print("Sending {Mi, Xi} to Service Provider...\n")
        return self.Mi, self.Xi

    def receive_authji(self, Authji: str):
        print("\n[Step 3: Smart Meter Verifies Service Provider]")
        k_prime = h(xor_bytes(xor_bytes(self.IDi_bytes, self.r1), self.r2))[:16]

        raw = base64.b64decode(Authji)
        iv = raw[:16]
        ciphertext = raw[16:]
        cipher = AES.new(k_prime, AES.MODE_CBC, iv)
        decrypted = unpad(cipher.decrypt(ciphertext), AES.block_size)

        hash_xor_r3 = decrypted[:len(self.r2)]
        hash_check = decrypted[len(self.r2):len(self.r2)+32]
        Mi_prime = decrypted[len(self.r2)+32:].decode()

        local_check = h(self.IDi_bytes + self.r1 + self.r2)
        if hash_check != local_check:
            print("Mutual authentication failed! Hash mismatch.")
            return None

        print("Mutual authentication passed.")

        r3 = xor_bytes(
            hash_xor_r3,
            h(xor_bytes(self.IDi_bytes, self.r2) + self.r1)[:len(self.r2)]
        )
        print(f"Recovered r3: {r3.hex()}")

        SKsm = h(self.IDi_bytes + self.r1 + self.r2 + r3)
        print(f"Session key (SKsm): {SKsm.hex()}")

        Authij = h(SKsm + r3).hex()
        print(f"Sending Authij to Service Provider:\n{Authij}\n")
        return Authij

    def verify_ackji(self, Ackji: str, r3: bytes, Mi_prime: str):
        print("[Step 5: Smart Meter Verifies Ackji]")
        expected_ack = h(xor_bytes(self.r2, r3) + self.r1).hex()

        if Ackji == expected_ack:
            print("Ackji verification successful. Finalizing session...")
            self.r1 = self.r2  # Update for next session
            self.Mi = Mi_prime
            print("Updated Tamper Resistant Device:")
            print({"IDi": self.IDi, "r1": self.r1.hex(), "Mi": self.Mi})
            return True
        else:
            print("Ackji verification failed! Terminating session.")
            return False



# ------------------------ RUN ------------------------

import time
import tracemalloc

tracemalloc.start()
start_time = time.time()

IDi = "SM001"
IDj = "SP001"
master_key = get_random_bytes(16)
print('master key',master_key.hex())

sp = ServiceProvider(IDj, master_key)
sm = SmartMeter(IDi)

# Step 1: Registration
sm.initiate_registration(sp)

# Step 2: Authentication
Mi, Xi = sm.initiate_authentication()
Authji = sp.process_authentication(Mi, Xi)

# Step 3: Smart Meter verifies Authji
if Authji:
    Authij = sm.receive_authji(Authji)

    # Step 4: SPj verifies Authij and sends Ackji
    if Authij:
        result = sp.verify_authij(Authij)
        if result:
            Ackji, r3, _, _, _, _, _, Mi_prime = result

            # Step 5: Smart Meter verifies Ackji
            sm.verify_ackji(Ackji, r3, Mi_prime)
            
end_time = time.time()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"\nTotal Execution Time: {end_time - start_time:.6f} seconds")
print(f"Peak Memory Usage: {peak / 1024:.2f} KB")

master key d64a115887458933d45039b70b2b2932
Registration complete for Smart Meter: SM001
Stored in Tamper Resistant Device:
{'IDi': 'SM001', 'r1': 'bd5216e2316e33b4', 'Mi': 'TANplVeaZDjNGvqfsFrpwrMdcb9T9BkzG+LVF7kINk4='}

[Step 1: Authentication Initiated]
Generated r2: 7f3470279b6db81b
Computed Xi = r2 ⊕ h(IDi || r1): a6e41e6fa55f65ac
Sending {Mi, Xi} to Service Provider...


[Step 2: Service Provider Processing Authentication Request]
Recovered IDi*: SM001
Recovered r1*: bd5216e2316e33b4
Authentication Passed: Qi matched.
Recovered r2': 7f3470279b6db81b
Generated r3: 5b41c5c4ddddefec
Session key (SKsp): 7008632ee51b0c01fc8d4717288e0ae0599ebc1fc7824cd67223ab51c515f0f7
Sending Authji to Smart Meter:
g3zKu2pn1wClEB7/aUhLA+p0H3BJyjmM//nTzmduS90RfQWm/LvyLpqHxtF24hrN9H9ffNOgvFCKKqFSkZuWB6jRqrbAb/3HyOZ+WmTMh1za7uDazDfsfKoaeTHmVnxtywu0QX3SFzRW96DCjRTheg==


[Step 3: Smart Meter Verifies Service Provider]
Mutual authentication passed.
Recovered r3: 5b41c5c4ddddefec
Session key (SKsm): 7008632